In [12]:
!pip install boto3 fabric

In [160]:
# Project settings

PROJECT_NAME = "OpenAssistants"

DOMAIN_NAME = "openassistants.io"

DJANGO_PROJECT_NAME = "oa"

REPO_URL = "https://github.com/rekognize/open-assistants-private"

BRANCHES = {
    'dev': f"test.{DOMAIN_NAME}",
    'main': DOMAIN_NAME
}

# Instance settings

SECURITY_GROUP_NAME = f"{PROJECT_NAME}-sg"
AMI_ID = "ami-0866a3c8686eaeeba" # Ubuntu Server 24.04 LTS
INSTANCE_TYPE = "t3.micro"
EBS_VOLUME_SIZE = 16  # GB
REGION = "us-east-1"
EC2_USER = "ubuntu"



In [210]:
import os
import json
import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv

load_dotenv()

s3_client = boto3.client('s3', region_name=REGION)

ec2_client = boto3.client('ec2', region_name=REGION)
ec2_resource = boto3.resource('ec2', region_name=REGION)


In [ ]:
# Create the EC2 instance

# Create the key pair

key_name = f"{PROJECT_NAME}-key"
key_path = os.path.expanduser(f"~/.ssh/{key_name}.pem")

try:
    key_pair = ec2_client.create_key_pair(KeyName=key_name)
except ClientError as e:
    if e.response['Error']['Code'] == "InvalidKeyPair.Duplicate":
        print(f"{key_name} already exists.")
    else:
        raise
else:
    with open(key_path, "w") as file:
        file.write(key_pair["KeyMaterial"])
    os.chmod(key_path, 0o400)


# Create and authorize the security group

try:
    response = ec2_client.create_security_group(
        GroupName=SECURITY_GROUP_NAME,
        Description="Security group for my project",
    )
except ClientError as e:
    if e.response['Error']['Code'] == "InvalidGroup.Duplicate":
        response = ec2_client.describe_security_groups(GroupNames=[SECURITY_GROUP_NAME])
        security_group_id = response['SecurityGroups'][0]['GroupId']
    else:
        raise e
else:
    security_group_id = response["GroupId"]
    ec2_client.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions=[
            {'IpProtocol': 'tcp', 'FromPort': 22, 'ToPort': 22,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
            {'IpProtocol': 'tcp', 'FromPort': 80, 'ToPort': 80,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
            {'IpProtocol': 'tcp', 'FromPort': 443, 'ToPort': 443,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        ]
    )

print("Launching EC2 instance...")
response = ec2_client.run_instances(
    ImageId=AMI_ID,
    InstanceType=INSTANCE_TYPE,
    KeyName=key_name,
    SecurityGroupIds=[security_group_id],
    MinCount=1,
    MaxCount=1,
    BlockDeviceMappings=[
        {
            "DeviceName": "/dev/sda1",
            "Ebs": {
                "VolumeSize": EBS_VOLUME_SIZE,
                "DeleteOnTermination": True,
                "VolumeType": "gp3"  # General Purpose SSD (gp3)
            },
        },
    ],
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [
                {
                    "Key": "Name",
                    "Value": PROJECT_NAME
                }
            ]
        }
    ]
)

# Get the instance ID of the newly created instance
instance_id = response["Instances"][0]["InstanceId"]
print("EC2 instance created with Instance ID:", instance_id)

# Wait for the instance to reach running state and retrieve the public IP and DNS
instance = ec2_resource.Instance(instance_id)

print("Waiting for the instance to enter 'running' state...")
instance.wait_until_running()
instance.reload()  # Refresh instance attributes

# Get the public IP and DNS for SSH
print("Instance is ready for SSH access.")
print("Public IP:", instance.public_ip_address)
print("Public DNS:", instance.public_dns_name)
print("Use the following command to SSH into the instance:")
print(f"ssh -i {key_path} ubuntu@{instance.public_ip_address}")

In [208]:
# Setup the EC2 instance with Fabric

from fabric import Connection

c = Connection(
    host=instance.public_ip_address, 
    user="ubuntu", 
    connect_kwargs={"key_filename": key_path},
)


In [ ]:
c.run("sudo apt update")
c.run("sudo apt upgrade -y")
c.run("sudo apt install -y python3-pip python3-venv uvicorn nginx git supervisor certbot python3-certbot-nginx")

In [ ]:
# Create and install the Github deploy key

import requests
from urllib.parse import urlparse

# Generate the deploy key with the repo name

deploy_key_path = f'/home/ubuntu/.ssh/{REPO_URL.split('/')[-1]}_deploy_key'
c.run(f'ssh-keygen -t rsa -b 2048 -f {deploy_key_path} -N ""')


# Install the key
github_token = os.getenv('GITHUB_TOKEN')  # GitHub token with repo access
public_key = c.run(f"cat {deploy_key_path}.pub").stdout.strip()

parsed_url = urlparse(REPO_URL)
repo_path = parsed_url.path.strip("/")
url = f"https://api.github.com/repos/{repo_path}/keys"

response = requests.post(
    url, 
    headers={
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3+json"
    }, 
    json={
        "title": f"{PROJECT_NAME} Deploy Key",
        "key": public_key,
        "read_only": True
    }
)


# Create the identity file

ssh_config_content = f"""
Host github.com
    IdentityFile {deploy_key_path}
    IdentitiesOnly yes
"""
c.run(f'echo "{ssh_config_content}" > /home/ubuntu/.ssh/config')
c.run("chmod 600 /home/ubuntu/.ssh/config")


In [ ]:
# Create socket dir for Uvicorn

socket_dir = f"/var/run/{PROJECT_NAME}"

c.sudo(f"mkdir -p {socket_dir}")
c.sudo(f"chown ubuntu:ubuntu {socket_dir}")
c.sudo(f"chmod 775 {socket_dir}")


In [216]:
# Branch specific section starts

BRANCH_NAME = "main"

In [ ]:
# Create the S3 bucket

S3_BUCKET_NAME = f'{PROJECT_NAME}-{BRANCH_NAME}'.lower()

s3_client.create_bucket(Bucket=S3_BUCKET_NAME)

s3_client.put_public_access_block(
    Bucket=S3_BUCKET_NAME,
    PublicAccessBlockConfiguration={
        "BlockPublicAcls": False,
        "IgnorePublicAcls": False,
        "BlockPublicPolicy": False,
        "RestrictPublicBuckets": False
    }
)

# Define the bucket policy; only give static public access, control media access with temp auth URLs
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{S3_BUCKET_NAME}/static/*"
        }
    ]
}

# Convert the policy to JSON string
bucket_policy_json = json.dumps(bucket_policy)

# Apply the policy to the bucket
try:
    response = s3_client.put_bucket_policy(Bucket=S3_BUCKET_NAME, Policy=bucket_policy_json)
    print(f"Policy successfully applied to bucket: {S3_BUCKET_NAME}")
except Exception as e:
    print(f"Error applying policy to bucket {S3_BUCKET_NAME}: {e}")


In [ ]:
# Server configuration

project_dir = f"/home/ubuntu/{PROJECT_NAME}-{BRANCH_NAME}"

# SSH URL for GitHub, using SSH key authentication
c.run(f"git clone -b {BRANCH_NAME} git@github.com:{repo_path}.git {project_dir}")

c.run(f"cd {project_dir} && python3 -m venv venv && source venv/bin/activate && pip install -r requirements.txt")


In [ ]:
# Copy the .env file for the environment
c.put(f".env-{BRANCH_NAME}", os.path.join(f"{project_dir}", ".env"))

In [ ]:
# Django migration
c.run(f"cd {project_dir} && source venv/bin/activate && python manage.py migrate")

In [ ]:
# Collect static
c.run(f"cd {project_dir} && source venv/bin/activate && python manage.py collectstatic --noinput")

In [ ]:
# Create superuser
superuser_name = f"{PROJECT_NAME}-admin"
superuser_email = f"admin@{DOMAIN_NAME}"
superuser_password = f"{PROJECT_NAME}-321*"

# Command to create superuser non-interactively (createsuperuser does not have non-interactive mode)
create_superuser_script = f"""
from django.contrib.auth import get_user_model

User = get_user_model()
if not User.objects.filter(username='{superuser_name}').exists():
    User.objects.create_superuser('{superuser_name}', '{superuser_email}', '{superuser_password}')
"""

# Run the Django shell command to create the superuser
c.run(f"cd {project_dir} && source venv/bin/activate && echo \"{create_superuser_script}\" | python manage.py shell")


In [ ]:
# Supervisor configuration for Uvicorn with Unix socket

socket_path = f"{socket_dir}/{BRANCH_NAME}.sock"

supervisor_conf = f"""
[program:{BRANCH_NAME}_uvicorn]
command={project_dir}/venv/bin/uvicorn {DJANGO_PROJECT_NAME}.asgi:application --uds {socket_path}
directory={project_dir}
user=ubuntu
autostart=true
autorestart=true
stdout_logfile=/var/log/{BRANCH_NAME}_uvicorn.log
stderr_logfile=/var/log/{BRANCH_NAME}_uvicorn_err.log
"""
supervisor_conf_path = f"/etc/supervisor/conf.d/{BRANCH_NAME}_uvicorn.conf"
c.run(f"echo '{supervisor_conf}' > /tmp/{BRANCH_NAME}_uvicorn.conf")
c.sudo(f"mv /tmp/{BRANCH_NAME}_uvicorn.conf {supervisor_conf_path}")
c.sudo("supervisorctl reread")
c.sudo("supervisorctl update")

# Restart Uvicorn via Supervisor
c.sudo(f"supervisorctl restart {BRANCH_NAME}_uvicorn")

In [ ]:
# Configure Nginx

domain = BRANCHES[BRANCH_NAME]

nginx_conf = f"""
server {{
    listen 80;
    server_name {domain};

    location /static/ {{
        alias {project_dir}/static;
    }}

    location /media/ {{
        alias {project_dir}/media;
    }}

    location / {{
        proxy_pass http://unix:{socket_path};
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }}
}}
"""

# Save and upload the nginx config file
nginx_conf_path = f"/etc/nginx/sites-available/{domain}"
c.run(f"echo '{nginx_conf}' > /tmp/{domain}_nginx.conf")
c.sudo(f"mv /tmp/{domain}_nginx.conf {nginx_conf_path}")
c.sudo(f"ln -sf {nginx_conf_path} /etc/nginx/sites-enabled/{domain}")
c.sudo("systemctl restart nginx")


In [ ]:
# Route53 DNS setup

# target_ip = instance.public_ip_address
target_ip = '3.87.35.129'

# Initialize the Route 53 client
route53 = boto3.client("route53", region_name=REGION)

# Fetch or create a hosted zone
def get_or_create_hosted_zone(domain):
    # List existing hosted zones
    response = route53.list_hosted_zones_by_name(DNSName=domain)
    zones = response.get("HostedZones", [])
    
    # Check if a hosted zone already exists
    for zone in zones:
        if zone["Name"].rstrip(".") == domain:
            return zone["Id"].split("/")[-1]  # Return the zone ID
    
    # Create a new hosted zone if not found
    response = route53.create_hosted_zone(
        Name=domain,
        CallerReference="unique-caller-reference-for-hosted-zone",  # Replace with a unique string
    )
    return response["HostedZone"]["Id"].split("/")[-1]

# Add or update DNS records
def create_dns_records(hosted_zone_id, subdomains, target_ip):
    changes = []
    for subdomain in subdomains:
        full_domain = f"{subdomain}.{DOMAIN_NAME}".strip(".")
        changes.append({
            "Action": "UPSERT",
            "ResourceRecordSet": {
                "Name": full_domain,
                "Type": "A",
                "TTL": 300,
                "ResourceRecords": [{"Value": target_ip}],
            },
        })
    
    # Apply the changes
    route53.change_resource_record_sets(
        HostedZoneId=hosted_zone_id,
        ChangeBatch={"Changes": changes},
    )
    print(f"DNS records updated for {subdomains}")


hosted_zone_id = get_or_create_hosted_zone(DOMAIN_NAME)

# "dev.openassistants.io" => "dev"
subdomains = [d[: -(len(DOMAIN_NAME) + 1)] for d in BRANCHES.values()]

create_dns_records(hosted_zone_id, subdomains, target_ip)


In [ ]:
# Create the Github workflow deploy.yml file

output_path = '.github/workflows/deploy.yml'

os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Generate the branches list in YAML format
branches_yaml = '\n'.join([f"      - {branch}" for branch in BRANCHES])

# Generate the conditional environment variable settings
conditional_run = ""
for i, branch in enumerate(BRANCHES):
    condition = "if" if i == 0 else "elif"
    conditional_run += f"""          {condition} [ "${{{{ github.ref_name }}}}" == "{branch}" ]; then
            echo "DEPLOY_DIR=OpenAssistants-{branch}" >> $GITHUB_ENV
            echo "ASGI_SERVER={branch}_uvicorn" >> $GITHUB_ENV
"""

# Close the if-elif blocks
conditional_run += "          fi"

# Complete YAML content
yaml_content = f"""name: Deploy to Server

on:
  push:
    branches:
{branches_yaml}

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
      - name: Checkout code
        uses: actions/checkout@v2

      - name: Set deployment parameters
        id: vars
        run: |
{conditional_run}

      - name: Add SSH Key
        run: |
          echo "${{{{ secrets.SSH_PRIVATE_KEY }}}}" > private_key.pem
          chmod 600 private_key.pem

      - name: Deploy to Server
        env:
          DEPLOY_DIR: ${{{{ env.DEPLOY_DIR }}}}
          ASGI_SERVER: ${{{{ env.ASGI_SERVER }}}}
        run: |
          ssh -i private_key.pem -o StrictHostKeyChecking=no ubuntu@{target_ip} <<EOF
            cd /home/ubuntu/$DEPLOY_DIR
            git fetch origin
            git reset --hard origin/${{{{ github.ref_name }}}}
            # Additional commands for Django:
            pip install -r requirements.txt
            python manage.py migrate
            python manage.py collectstatic --noinput
            sudo supervisorctl restart $ASGI_SERVER
          EOF
"""

# Write the YAML content to the file
with open(output_path, 'w') as f:
    f.write(yaml_content)

print(f"`deploy.yml` has been created at `{output_path}`")


In [ ]:
# Certbot HTTPS

for domain in BRANCHES.values():
    certbot_command = f"certbot --nginx -d {domain} --non-interactive --agree-tos --email admin@{domain}"
    c.sudo(certbot_command)

c.sudo("nginx -t")
c.sudo("systemctl reload nginx")


In [218]:
# AWS SES setup

ses = boto3.client("ses")

# Get domain verification token
response = ses.verify_domain_identity(Domain=DOMAIN_NAME)
verification_token = response['VerificationToken']

# Add the TXT record
response = route53.change_resource_record_sets(
    HostedZoneId=hosted_zone_id,
    ChangeBatch={
        "Changes": [
            {
                "Action": "UPSERT",
                "ResourceRecordSet": {
                    "Name": f"_amazonses.{DOMAIN_NAME}",
                    "Type": "TXT",
                    "TTL": 300,
                    "ResourceRecords": [{"Value": f'"{verification_token}"'}],
                },
            }
        ]
    },
)


In [222]:
response = ses.get_identity_verification_attributes(
    Identities=[DOMAIN_NAME]
)



In [ ]:
# Check SES domain verification status

is_verified = False

response = ses.get_identity_verification_attributes(
    Identities=[DOMAIN_NAME]
)
attributes = response["VerificationAttributes"].get(DOMAIN_NAME)
if attributes:
    is_verified = attributes["VerificationStatus"] == 'Success'

if is_verified:
    print(f"{DOMAIN_NAME} has been verified")